### Imports

In [ ]:
import sqlite3 as sql
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder

conn = sql.connect("states.db")
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(tables_query, conn)
table_names = tables_df['name'].tolist()[1:]


dataframes = {}
for table_name in table_names:
    dataframes[table_name] = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

In [154]:
unemployment_rate = dataframes['unemployment_rate']
state_survival_rates = dataframes['survival_rates']
industry_survival_rates = dataframes['industry_survival_rates']
establishments = dataframes['establishments']

### Data reformatting

Reformat into 1530 samples for state data

In [59]:
state_survival_rates_filtered = state_survival_rates[state_survival_rates["Year Established"] != state_survival_rates["Year"]]
state_survival_rates_filtered["Year"] = state_survival_rates_filtered["Year"].astype('int')
state_survival_rates_filtered["Survival Rates of Previous Year's Survivors"] = state_survival_rates_filtered["Survival Rates of Previous Year's Survivors"].astype('float')
state_survival_rates_filtered['Surviving Establishments'] = state_survival_rates_filtered['Surviving Establishments'].str.replace(',', '').astype(int)
state_survival_rates_grouped = state_survival_rates_filtered.groupby(["Year", "State"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))

/var/folders/m7/6v1zwk1505d2ph7h2jch6t340000gn/T/ipykernel_47581/1002942169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_survival_rates_filtered["Year"] = state_survival_rates_filtered["Year"].astype('int')
/var/folders/m7/6v1zwk1505d2ph7h2jch6t340000gn/T/ipykernel_47581/1002942169.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_survival_rates_filtered["Survival Rates of Previous Year's Survivors"] = state_survival_rates_filtered["Survival Rates of Previous Year's Survivors"].astype('

Get reformatted industry data

In [61]:
industry_survival_rates_filtered = industry_survival_rates[industry_survival_rates["Year Established"] != industry_survival_rates["Year"]]
industry_survival_rates_filtered["Year"] = industry_survival_rates_filtered["Year"].astype('int')
industry_survival_rates_filtered["Survival Rates of Previous Year's Survivors"] = industry_survival_rates_filtered["Survival Rates of Previous Year's Survivors"].astype('float')
industry_survival_rates_filtered['Surviving Establishments'] = industry_survival_rates_filtered['Surviving Establishments'].str.replace(',', '').astype(int)
industry_survival_rates_grouped = industry_survival_rates_filtered.groupby(["Year", "Industry"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))

/var/folders/m7/6v1zwk1505d2ph7h2jch6t340000gn/T/ipykernel_47581/1975599014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_survival_rates_filtered["Year"] = industry_survival_rates_filtered["Year"].astype('int')
/var/folders/m7/6v1zwk1505d2ph7h2jch6t340000gn/T/ipykernel_47581/1975599014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry_survival_rates_filtered["Survival Rates of Previous Year's Survivors"] = industry_survival_rates_filtered["Survival Rates of Previous Year's Survivor

In [ ]:
total_survival_rate = pd.concat([state_survival_rates_filtered, industry_survival_rates_filtered])
total_survival_rate_grouped = total_survival_rate.groupby(["Year"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))

We want to multiply each combination of state and industry, then divide by that year

In [ ]:
results = []
for (year, state), state_val in state_survival_rates_grouped.items():
    for (ind_year, industry), ind_val in industry_survival_rates_grouped.items():
        if year == ind_year:
            result = (year, state, industry), (state_val * ind_val) / total_survival_rate_grouped[year]
            results.append(result)

result = pd.Series(dict(results))
unemployment_rate["Year"] = unemployment_rate["Year"].astype('int')

In [118]:
state_df = state_survival_rates_grouped.unstack()
industry_df = industry_survival_rates_grouped.unstack()

In [171]:
numerical_state = dict(zip(state_df.columns, range(len(state_df.columns))))
numerical_state_rev = dict(zip(range(len(state_df.columns)), state_df.columns))
numerical_industry = dict(zip(industry_df.columns, range(len(industry_df.columns))))
numerical_industry_rev = dict(zip(range(len(industry_df.columns)), industry_df.columns))

In [169]:
pre_df = []
for year in result.index.get_level_values(0).unique():
    for state in state_df.columns:
        unemployment = unemployment_rate[(unemployment_rate["Year"] == year) & (unemployment_rate["State"] == state)]["Unemployment Rate"]
        for industry in industry_df.columns:
            response = result[year, state, industry]
            pre_df.append((year, numerical_state[state], numerical_industry[industry], float(unemployment.iloc[0]), response))
final_dataset = pd.DataFrame(pre_df, columns=["Year", "State", "Industry", "Unemployment Rate", "Response"])
             

### Model

In [202]:
class EconDataset(Dataset):
    def __init__(self, data, state_col, industry_col, unemployment_col, response_col):
        self.data = data.copy()
        self.state_col = state_col
        self.industry_col = industry_col
        self.unemployment_col = unemployment_col
        self.response_col = response_col
        self.state_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.industry_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self._preprocess_data()

    def _preprocess_data(self):
        self.encoded_states = self.state_encoder.fit_transform(self.data[[self.state_col]]) # Applying one-hot encoding to the state column
        self.encoded_industries = self.industry_encoder.fit_transform(self.data[[self.industry_col]]) # Applying one-hot encoding to the industry
        self.unemployment_stats = self.data[self.unemployment_col].values.reshape(-1, 1) # Turns into column vector
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        state = self.encoded_states[idx]
        industry = self.encoded_industries[idx]
        unemployment = self.unemployment_stats[idx]
        predictor = np.concatenate((unemployment, state, industry), axis=0)
        response = self.data[self.response_col].values[idx]
        return torch.tensor(predictor, dtype=torch.float32), torch.tensor(response, dtype=torch.float32).view(1) #Response is reshaped to a column vector.

In [199]:
SurvivalData = CustomDataset(final_dataset, "State", "Industry", "Unemployment Rate", "Response")
dataloader = DataLoader(SurvivalData, batch_size=2, shuffle=True)  # shuffle = True is important for training


In [214]:
class SurvivalRateModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size = 1):
        super(SurvivalRateModel, self).__init__()

        # Define layers:
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, output_size)
        )

    def forward(self, x):
        return self.layers(x)

    
    def train_step(self, x, y, criterion, optimizer):
        optimizer.zero_grad()
        outputs = self(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        return loss.item()
    
    def train_loop(self, dataloader, num_epochs=10, learning_rate=0.001, device = "cpu"):
        self.to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        self.train() #set model to train mode.
        for epoch in range(num_epochs):
            total_loss = 0
            for x, y in dataloader:
                x, y = x.to(device), y.to(device)
                loss = self.train_step(x, y, criterion, optimizer)
                total_loss += loss
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')
    

In [215]:
data = EconDataset(final_dataset, "State", "Industry", "Unemployment Rate", "Response")
dataloader = DataLoader(data, batch_size=2, shuffle=True)
Model = SurvivalRateModel(input_size=1 + len(data.state_encoder.categories_[0]) + len(data.industry_encoder.categories_[0]), hidden_size1=4, hidden_size2=8)

In [216]:
Model.train_loop(dataloader, num_epochs=10, learning_rate=0.001, device="cpu")

Epoch [1/10], Loss: 1980.7699
Epoch [2/10], Loss: 16.1663
Epoch [3/10], Loss: 16.1649
Epoch [4/10], Loss: 16.1720
Epoch [5/10], Loss: 16.1659
Epoch [6/10], Loss: 16.1555
Epoch [7/10], Loss: 16.1643
Epoch [8/10], Loss: 16.1726
Epoch [9/10], Loss: 16.1633
Epoch [10/10], Loss: 16.1603
